In [ ]:
import numpy as np
import itertools
import math
import random
import matplotlib.pyplot as plt
import scipy.optimize as opt
from numpy import linalg


# Heaveside Step Function. # More similar to Kronecker Delta Function. 

def heaviside_step_function(x):

    if x > 0:
        return 1
    else:
        return 0

# Function for Rectangular Approximation. 

def rectangular_approximation(N, n_p):
    
    # Fixing Grid Space.

    int_points = int(math.pow(n_p, 1/N)) # Integer points to set the grid. 

    set_grid, arr_grid, grid_point = np.linspace(0, 1, int_points), [], []
    
    arr1 = itertools.combinations_with_replacement(set_grid, N)
    
    for i in list(arr1):
        k1 = set(itertools.permutations(i))
        arr_grid = np.append(arr_grid, k1)
    
    l1 = len(arr_grid)
    
    for i in range(l1):
        for j in(arr_grid[i]):
            grid_point = np.append(grid_point,j)
            
    l2 = len(grid_point)
    
    sum_res = 0
    
    for i in grid_point:
        norm = linalg.norm(i)
        sum_res = sum_res + heaviside_step_function(1-norm)
    
    volume_approx = pow(2,N) * (sum_res / l2)
    
    return volume_approx

# Function for Monte Carlo Integration

def mc_int(N, n_p, seed):

    set_grid, int_points = [], int(math.pow(n_p, 1/N))

    for i in range(N):

      random.seed(seed)

      for j in range(int_points):
          set_grid.append(random.uniform(0, 1))

    arr2 = np.array(set_grid)

    set_grid = np.reshape(arr2 , (N, int_points))
  
    sum_res = 0


    for i in range(int_points):
      
        k2 = linalg.norm(set_grid[:,i])
        sum_res = sum_res + heaviside_step_function(1 - k2)

    volume_approx = math.pow(2,N) * (sum_res / int_points)
    return volume_approx

# Function giving Analytical Solution.

def analytical_soln(N):

    k4 = 0

    if N > 1:
        k4 =  ( 2*np.pi / N) * analytical_soln(N-2)
    elif N == 0:
        k4 = 1
    elif N == 1:
        k4 = 2

    V = np.zeros(N+1)
    res_vol = k4
    
    return res_vol


# Power Law Function for the 

def linear_func(a, x, k):
    return a * pow(x,k)



# Function Calculating Error in the Rectangular Approximation.

def rectangular_approx_error(N, n_p):

    for i in N:
        rect_volume = []
        for j in n_p:
            rect_volume.append(rectangular_approximation(i, j))
        exact_volume = analytical_soln(i)
        errors = np.abs(np.array(rect_volume) - exact_volume)
        param, pcov = opt.curve_fit(linear_func, n_p, errors)
        
        plt.plot(n_p, errors, label = "N = {}".format(i))
        print("Dimension: %s; Power-law exponent = %s, k = %s" %(i, param[0], param[1]))


    plt.xlabel("$n_p$")
    plt.xscale('log')
    plt.ylabel("Error")
    plt.yscale('log')
    plt.title("Error in rectangular approximation.")
    plt.show()


# Function quantifying the Monte Carlo Variance.

def mc_var(N, n_p):

    for i in N:
        variance = []
        for j in n_p:
            monte_volume = []
            for k in range(20):
                monte_volume.append(mc_int(i, j, k))
            exp, var = np.mean(monte_volume), np.std(monte_volume)
            variance.append(var)
        param, pcov = opt.curve_fit(linear_func, n_p, variance)
        plt.plot(n_p, np.array(variance), label="N={}".format(i))
        print("Dimension: %s; Power-law exponent = %s, k = %s" %(i, param[0], param[1]))


    plt.xlabel("$n_p$")
    plt.xscale('log')
    plt.ylabel("$\sigma^2$")
    plt.yscale('log')
    plt.title("$\sigma^2$ as a function of integration points $n_p$")
    plt.show()

In [ ]:
N = [2, 3, 4, 5, 6]
 
n_p = np.array([pow(3,2), pow(5,5), pow(6,4), pow(6,5)])


rectangular_approx_error(N, n_p)
mc_var(N, n_p)




#Report



#### Volume of a Hypersphere

The volume of a hypersphere is calculated by method and formulas suggested in the lab notes. The the hyperradius is taken to be 1 till this project.

#### Rectangular Approximation

For the rectangular approximation, the grid is set and at every grid point, the function is multiplied with the norm of the grid-interval and is then summed up. The process is very memory consuming and takes a while to produce results.

The error plot shows that the error increases for increasing dimensions along with extra addition of the number of grid-points.

#### Monte-Carlo Approximation

In order to calulate the probability distribution inside the hypersphere, random grid points were set and the heaveside function was called to calculate the points. From these points the volume was calculated. The variance was calulated and plotted and decreasing linear relation was observed as per the theory prediciton.

#### Discussion

Both process is computationally demanding in terms of memory consumption and the computational time. 

Using power law to compare the error and variance of the Rectangular and Monte Carlo Method. For the Monte Carlo Method larger the dimension is, larger the constant is choosen and thus giving smaller variance but for the Rectangular Approxiamtion the constant remains same, thus error is constantly same. 
